In [ ]:
# 🧹 곱창집 입지 분석 데이터 정제
# 02-1_data_cleaning.ipynb
# 
# 목적: 마스터 데이터셋의 품질 문제 해결
#       - 업종별 중복 제거 (471K → 1.6K)
#       - 좌표 정보 복구
#       - 정확한 상권별 집계

# ================================================================================
# 📚 라이브러리 임포트
# ================================================================================

import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
from pathlib import Path
from datetime import datetime

# 설정
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

print("🧹 곱창집 입지 분석 데이터 정제 시작!")
print(f"⏰ 시작 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 60)

# ================================================================================
# 📂 원본 데이터 다시 로드 (정제용)
# ================================================================================

print("📂 원본 데이터 다시 로드 (정제용)")
print("-" * 30)

# 데이터 경로
DATA_PATH = Path("../data")
RAW_PATH = DATA_PATH / "raw"

# 1. 기준 데이터: 상권 경계 (1,650개 상권)
boundary_df = pd.read_csv(DATA_PATH / "boundary/boundary_all.csv", encoding='utf-8')
print(f"📍 상권 경계: {boundary_df.shape} (기준)")

# 2. 최신 분기 데이터들 로드
flow_df = pd.read_csv(RAW_PATH / "flow/flow_all.csv", encoding='utf-8')
latest_quarter = flow_df['기준_년분기_코드'].max()
flow_latest = flow_df[flow_df['기준_년분기_코드'] == latest_quarter].copy()
print(f"👥 유동인구 ({latest_quarter}): {flow_latest.shape}")

# 3. 매출 데이터 (문제의 핵심)
sales_df = pd.read_csv(RAW_PATH / "sales/sales_all.csv", encoding='utf-8')
sales_latest = sales_df[sales_df['기준_년분기_코드'] == latest_quarter].copy()
print(f"💰 매출 원본 ({latest_quarter}): {sales_latest.shape} ← 문제!")

# 4. 상주인구
resident_df = pd.read_csv(RAW_PATH / "resident/resident_all.csv", encoding='utf-8')
print(f"🏠 상주인구: {resident_df.shape}")

# 5. 소득·소비
income_df = pd.read_csv(RAW_PATH / "income_exp/income_all.csv", encoding='utf-8')
income_latest = income_df[income_df['기준_년분기_코드'] == latest_quarter].copy()
print(f"💳 소득·소비 ({latest_quarter}): {income_latest.shape}")

print("\n" + "=" * 60)

# ================================================================================
# 🔍 문제 진단: 중복 데이터 분석
# ================================================================================

print("🔍 문제 진단: 중복 데이터 분석")
print("-" * 30)

# 1. 매출 데이터 중복 원인 분석
print("1️⃣ 매출 데이터 구조 분석:")
print(f"   📊 총 행수: {len(sales_latest):,}개")
print(f"   🏪 고유 상권수: {sales_latest['상권_코드'].nunique():,}개")

# 업종별 분석
if '서비스_업종_코드_명' in sales_latest.columns:
    service_counts = sales_latest['서비스_업종_코드_명'].value_counts()
    print(f"   🏷️ 업종 종류: {len(service_counts)}개")
    print(f"   📋 주요 업종:")
    for i, (service, count) in enumerate(service_counts.head(5).items()):
        print(f"      {i+1}. {service}: {count:,}개")

# 상권별 행수 분석
district_counts = sales_latest['상권_코드'].value_counts()
print(f"\n   📊 상권별 평균 행수: {district_counts.mean():.1f}개")
print(f"   📊 상권별 최대 행수: {district_counts.max()}개")

# 2. 기타 데이터 중복 확인
print(f"\n2️⃣ 기타 데이터 중복 확인:")
datasets = {
    'boundary': boundary_df,
    'flow_latest': flow_latest,
    'resident': resident_df,
    'income_latest': income_latest
}

for name, df in datasets.items():
    if '상권_코드' in df.columns:
        unique_districts = df['상권_코드'].nunique()
        total_rows = len(df)
        print(f"   {name:12s}: {total_rows:5,}행 / {unique_districts:4,}개 상권 = {total_rows/unique_districts if unique_districts > 0 else 0:4.1f}행/상권")

print("\n" + "=" * 60)

# ================================================================================
# 🔧 데이터 정제 1단계: 업종별 집계
# ================================================================================

print("🔧 데이터 정제 1단계: 업종별 집계")
print("-" * 30)

# 1. 매출 데이터 상권별 집계
print("1️⃣ 매출 데이터 상권별 집계:")

# 매출 관련 컬럼 식별
sales_amount_cols = [col for col in sales_latest.columns if '매출_금액' in col]
sales_count_cols = [col for col in sales_latest.columns if '매출_건수' in col]

print(f"   📋 매출금액 컬럼: {len(sales_amount_cols)}개")
print(f"   📋 매출건수 컬럼: {len(sales_count_cols)}개")

# 상권별 집계 (업종별 합계)
sales_agg_dict = {}

# 매출금액 합계
for col in sales_amount_cols:
    sales_agg_dict[col] = 'sum'

# 매출건수 합계  
for col in sales_count_cols:
    sales_agg_dict[col] = 'sum'

# 기본 정보는 첫 번째 값 사용
basic_cols = ['상권_구분_코드', '상권_구분_코드_명', '상권_코드_명']
for col in basic_cols:
    if col in sales_latest.columns:
        sales_agg_dict[col] = 'first'

# 집계 실행
sales_clean = sales_latest.groupby('상권_코드').agg(sales_agg_dict).reset_index()

print(f"   📊 집계 전: {len(sales_latest):,}행")
print(f"   📊 집계 후: {len(sales_clean):,}행")
print(f"   ✅ 상권 수 일치: {len(sales_clean) == sales_latest['상권_코드'].nunique()}")

# 2. 기타 데이터도 중복 제거 (혹시 모를 중복)
print(f"\n2️⃣ 기타 데이터 중복 제거:")

# 유동인구 (이미 깨끗해야 하지만 확인)
flow_clean = flow_latest.drop_duplicates(subset=['상권_코드']).copy()
print(f"   👥 유동인구: {len(flow_latest):,} → {len(flow_clean):,}개")

# 상주인구 - 상권별 집계 (연령대별 합계)
resident_pop_cols = [col for col in resident_df.columns if '상주인구_수' in col]
if not resident_pop_cols:
    resident_pop_cols = [col for col in resident_df.columns if '인구_수' in col]

if resident_pop_cols:
    resident_agg = {}
    for col in resident_pop_cols:
        resident_agg[col] = 'sum'
    
    # market_code 기준 집계
    if 'market_code' in resident_df.columns:
        resident_clean = resident_df.groupby('market_code').agg(resident_agg).reset_index()
        resident_clean['상권_코드'] = resident_clean['market_code']  # 컬럼명 통일
    else:
        resident_clean = resident_df.copy()
        
    print(f"   🏠 상주인구: {len(resident_df):,} → {len(resident_clean):,}개")
else:
    resident_clean = resident_df.copy()
    print(f"   🏠 상주인구: 집계 컬럼 없음 ({len(resident_clean):,}개 유지)")

# 소득·소비 중복 제거
income_clean = income_latest.drop_duplicates(subset=['상권_코드']).copy()
print(f"   💳 소득·소비: {len(income_latest):,} → {len(income_clean):,}개")

print("\n" + "=" * 60)

# ================================================================================
# 🗺️ 데이터 정제 2단계: 좌표 정보 복구
# ================================================================================

print("🗺️ 데이터 정제 2단계: 좌표 정보 복구")
print("-" * 30)

# 1. 좌표 컬럼 재탐지
print("1️⃣ 좌표 컬럼 재탐지:")
coord_columns = [col for col in boundary_df.columns if any(keyword in col.lower() 
                for keyword in ['x', 'y', '경도', '위도', 'lng', 'lat', '좌표'])]

print(f"   📍 좌표 관련 컬럼: {coord_columns}")

# 좌표 데이터 확인
for col in coord_columns:
    sample_values = boundary_df[col].dropna().head(5).tolist()
    print(f"      {col}: {sample_values}")

# 2. 경도/위도 식별
lng_col = None
lat_col = None

# 경도 (X좌표, 동경)
for col in coord_columns:
    if any(keyword in col.lower() for keyword in ['x', '경도', 'lng', 'lon']):
        lng_col = col
        break

# 위도 (Y좌표, 북위)  
for col in coord_columns:
    if any(keyword in col.lower() for keyword in ['y', '위도', 'lat']):
        lat_col = col
        break

if lng_col and lat_col:
    print(f"\n   ✅ 경도 컬럼: {lng_col}")
    print(f"   ✅ 위도 컬럼: {lat_col}")
    
    # 좌표 유효성 검증
    valid_lng = boundary_df[lng_col].notna()
    valid_lat = boundary_df[lat_col].notna()
    valid_coords = valid_lng & valid_lat
    
    print(f"   📊 유효한 경도: {valid_lng.sum():,}개")
    print(f"   📊 유효한 위도: {valid_lat.sum():,}개") 
    print(f"   📊 완전한 좌표: {valid_coords.sum():,}개")
    
    # 좌표 범위 검증 (서울 지역)
    if valid_coords.sum() > 0:
        lng_range = [boundary_df.loc[valid_coords, lng_col].min(), 
                     boundary_df.loc[valid_coords, lng_col].max()]
        lat_range = [boundary_df.loc[valid_coords, lat_col].min(), 
                     boundary_df.loc[valid_coords, lat_col].max()]
        
        print(f"   📐 경도 범위: {lng_range[0]:.6f} ~ {lng_range[1]:.6f}")
        print(f"   📐 위도 범위: {lat_range[0]:.6f} ~ {lat_range[1]:.6f}")
        
        # 서울 좌표 범위 검증 (대략적)
        seoul_lng_range = [126.7, 127.3]  # 서울 경도 범위
        seoul_lat_range = [37.4, 37.7]    # 서울 위도 범위
        
        lng_valid = (lng_range[0] >= seoul_lng_range[0] and lng_range[1] <= seoul_lng_range[1])
        lat_valid = (lat_range[0] >= seoul_lat_range[0] and lat_range[1] <= seoul_lat_range[1])
        
        if lng_valid and lat_valid:
            print(f"   ✅ 서울 지역 좌표 범위 적합")
        else:
            print(f"   ⚠️ 좌표 범위 확인 필요 (서울 지역 벗어남)")
else:
    print(f"   ❌ 경도/위도 컬럼을 찾을 수 없음")

print("\n" + "=" * 60)

# ================================================================================
# 🔗 데이터 정제 3단계: 깔끔한 마스터 데이터셋 재구축
# ================================================================================

print("🔗 데이터 정제 3단계: 깔끔한 마스터 데이터셋 재구축")
print("-" * 30)

# 1. 기준 데이터: 상권 경계 (좌표 포함)
print("1️⃣ 기준 데이터 설정:")
master_clean = boundary_df.copy()

# 상권코드 컬럼 통일
if '상권_코드' in master_clean.columns:
    master_clean['TRDAR_CD'] = master_clean['상권_코드']
else:
    print("   ❌ 상권_코드 컬럼 없음!")

print(f"   📍 기준 상권: {len(master_clean):,}개")

# 2. 유동인구 조인 (17-24시 야간 계산 포함)
print(f"\n2️⃣ 유동인구 데이터 조인:")

# 야간 유동인구 재계산
def calculate_night_flow_clean(df):
    """17-24시 야간 유동인구 계산 (깔끔한 버전)"""
    df = df.copy()
    
    # 야간 유동인구 (17-24시)
    df['야간_유동인구_17_24'] = (df['시간대_17_21_유동인구_수'] + 
                               df['시간대_21_24_유동인구_수'])
    
    # 야간 비율
    df['야간_비율'] = df['야간_유동인구_17_24'] / df['총_유동인구_수']
    
    # 주말 vs 주중
    df['주말_평균'] = (df['토요일_유동인구_수'] + df['일요일_유동인구_수']) / 2
    df['주중_평균'] = (df['월요일_유동인구_수'] + df['화요일_유동인구_수'] + 
                     df['수요일_유동인구_수'] + df['목요일_유동인구_수'] + 
                     df['금요일_유동인구_수']) / 5
    df['주말_프리미엄'] = df['주말_평균'] / df['주중_평균']
    
    return df

flow_clean = calculate_night_flow_clean(flow_clean)

# 상권코드 통일 후 조인
flow_clean['TRDAR_CD'] = flow_clean['상권_코드']

# 핵심 컬럼만 선택
flow_key_cols = ['TRDAR_CD', '총_유동인구_수', '야간_유동인구_17_24', '야간_비율',
                 '주말_평균', '주중_평균', '주말_프리미엄',
                 '남성_유동인구_수', '여성_유동인구_수',
                 '연령대_20_유동인구_수', '연령대_30_유동인구_수', '연령대_40_유동인구_수']

available_flow_cols = [col for col in flow_key_cols if col in flow_clean.columns]
flow_selected = flow_clean[available_flow_cols].copy()

master_clean = master_clean.merge(flow_selected, on='TRDAR_CD', how='left')
print(f"   📊 유동인구 매칭: {(master_clean['총_유동인구_수'].notna().sum() / len(master_clean)):.1%}")

# 3. 매출 데이터 조인 (집계된 깔끔한 데이터)
print(f"\n3️⃣ 매출 데이터 조인:")
sales_clean['TRDAR_CD'] = sales_clean['상권_코드']

# 주요 매출 컬럼 선택
main_sales_col = '당월_매출_금액' if '당월_매출_금액' in sales_clean.columns else sales_amount_cols[0]
sales_clean['총_매출'] = sales_clean[main_sales_col]

sales_selected = sales_clean[['TRDAR_CD', '총_매출']].copy()
master_clean = master_clean.merge(sales_selected, on='TRDAR_CD', how='left')
print(f"   📊 매출 매칭: {(master_clean['총_매출'].notna().sum() / len(master_clean)):.1%}")

# 4. 상주인구 조인
print(f"\n4️⃣ 상주인구 조인:")
if 'market_code' in resident_clean.columns:
    resident_clean['TRDAR_CD'] = resident_clean['market_code']
elif '상권_코드' in resident_clean.columns:
    resident_clean['TRDAR_CD'] = resident_clean['상권_코드']

# 총 상주인구 계산
resident_pop_cols = [col for col in resident_clean.columns if '인구_수' in col]
if len(resident_pop_cols) > 1:
    resident_clean['총_상주인구'] = resident_clean[resident_pop_cols].sum(axis=1)
elif len(resident_pop_cols) == 1:
    resident_clean['총_상주인구'] = resident_clean[resident_pop_cols[0]]
else:
    print("   ⚠️ 상주인구 컬럼 없음")
    resident_clean = None

if resident_clean is not None:
    resident_selected = resident_clean[['TRDAR_CD', '총_상주인구']].copy()
    master_clean = master_clean.merge(resident_selected, on='TRDAR_CD', how='left')
    print(f"   📊 상주인구 매칭: {(master_clean['총_상주인구'].notna().sum() / len(master_clean)):.1%}")

# 5. 소득 데이터 조인
print(f"\n5️⃣ 소득 데이터 조인:")
income_clean['TRDAR_CD'] = income_clean['상권_코드']

income_cols = [col for col in income_clean.columns if '소득' in col]
if income_cols:
    income_clean['평균_소득'] = income_clean[income_cols[0]]
    income_selected = income_clean[['TRDAR_CD', '평균_소득']].copy()
    master_clean = master_clean.merge(income_selected, on='TRDAR_CD', how='left')
    print(f"   📊 소득 매칭: {(master_clean['평균_소득'].notna().sum() / len(master_clean)):.1%}")

print(f"\n🎯 깔끔한 마스터 데이터셋: {master_clean.shape}")
print("\n" + "=" * 60)

# ================================================================================
# 📊 곱창집 특화 파생지표 재계산
# ================================================================================

print("📊 곱창집 특화 파생지표 재계산")
print("-" * 30)

# 1. 기본 효율성 지표
print("1️⃣ 기본 효율성 지표:")

# 총 인구 (상주 + 유동)
if '총_상주인구' in master_clean.columns:
    master_clean['총_인구'] = (master_clean['총_유동인구_수'].fillna(0) + 
                              master_clean['총_상주인구'].fillna(0))
else:
    master_clean['총_인구'] = master_clean['총_유동인구_수'].fillna(0)

# 매출 효율성
if '총_매출' in master_clean.columns:
    master_clean['인당_매출효율'] = master_clean['총_매출'] / master_clean['총_인구']
    master_clean['야간_매출효율'] = master_clean['총_매출'] / master_clean['야간_유동인구_17_24']
    
    # 무한대 값 처리
    master_clean['인당_매출효율'] = master_clean['인당_매출효율'].replace([np.inf, -np.inf], np.nan)
    master_clean['야간_매출효율'] = master_clean['야간_매출효율'].replace([np.inf, -np.inf], np.nan)
    
    print(f"   ✅ 인당 매출효율: 평균 {master_clean['인당_매출효율'].mean():.0f}원/명")
    print(f"   ✅ 야간 매출효율: 평균 {master_clean['야간_매출효율'].mean():.0f}원/명")

# 2. 곱창집 특화 지표
print(f"\n2️⃣ 곱창집 특화 지표:")

# 주류 친화 지수
master_clean['주류친화_지수'] = (master_clean['야간_비율'].fillna(0) * 0.6 + 
                              (master_clean['주말_프리미엄'].fillna(1) - 1) * 0.4)

# 타겟 연령대 비율 (20-40대)
target_age_cols = ['연령대_20_유동인구_수', '연령대_30_유동인구_수', '연령대_40_유동인구_수']
available_age_cols = [col for col in target_age_cols if col in master_clean.columns]

if available_age_cols:
    master_clean['타겟연령_비율'] = (master_clean[available_age_cols].sum(axis=1) / 
                                 master_clean['총_유동인구_수'])
    master_clean['타겟연령_비율'] = master_clean['타겟연령_비율'].fillna(0)
    print(f"   ✅ 타겟연령(20-40대) 비율: 평균 {master_clean['타겟연령_비율'].mean():.1%}")

# 성비 균형도
if all(col in master_clean.columns for col in ['남성_유동인구_수', '여성_유동인구_수']):
    master_clean['성비_균형도'] = (1 - abs(master_clean['남성_유동인구_수'] - 
                                        master_clean['여성_유동인구_수']) / 
                                  master_clean['총_유동인구_수'])
    master_clean['성비_균형도'] = master_clean['성비_균형도'].fillna(0.5)
    print(f"   ✅ 성비 균형도: 평균 {master_clean['성비_균형도'].mean():.1%}")

print(f"   ✅ 주류친화 지수: 평균 {master_clean['주류친화_지수'].mean():.2f}")

# 3. 점수화 (1-5점)
print(f"\n3️⃣ 점수화 (1-5점):")

scoring_cols = ['야간_유동인구_17_24', '총_매출', '주류친화_지수']
available_scoring_cols = [col for col in scoring_cols if col in master_clean.columns]

for col in available_scoring_cols:
    score_col = f'{col}_점수'
    try:
        master_clean[score_col] = pd.qcut(master_clean[col].fillna(0), 
                                         q=5, labels=[1,2,3,4,5], 
                                         duplicates='drop').astype(float)
        print(f"   ✅ {col} → {score_col}")
    except Exception as e:
        print(f"   ⚠️ {col} 점수화 실패: {e}")

print("\n" + "=" * 60)

# ================================================================================
# 🗺️ 지리정보 GeoDataFrame 생성
# ================================================================================

print("🗺️ 지리정보 GeoDataFrame 생성")
print("-" * 30)

if lng_col and lat_col:
    try:
        import geopandas as gpd
        from shapely.geometry import Point
        
        # 유효한 좌표 데이터 필터링
        valid_coords = (master_clean[lng_col].notna() & 
                       master_clean[lat_col].notna())
        
        geo_data = master_clean[valid_coords].copy()
        
        # Point 지오메트리 생성
        geometry = [Point(lng, lat) for lng, lat in 
                   zip(geo_data[lng_col], geo_data[lat_col])]
        
        # GeoDataFrame 생성
        master_gdf = gpd.GeoDataFrame(geo_data, geometry=geometry, crs='EPSG:4326')
        
        print(f"   ✅ GeoDataFrame 생성: {len(master_gdf):,}개 상권")
        print(f"   📐 좌표계: EPSG:4326 (WGS84)")
        print(f"   📊 지리정보 비율: {len(master_gdf)/len(master_clean):.1%}")
        
        # GPKG 저장
        output_path = DATA_PATH / "processed"
        output_path.mkdir(exist_ok=True)
        
        gpkg_file = output_path / "seoul_gopchang_master_clean.gpkg"
        master_gdf.to_file(gpkg_file, driver='GPKG')
        print(f"   💾 GeoPackage 저장: {gpkg_file}")
        
    except Exception as e:
        print(f"   ❌ GeoDataFrame 생성 실패: {e}")
        master_gdf = None
else:
    print(f"   ⚠️ 좌표 정보 부족 - GeoPackage 생성 불가")
    master_gdf = None

# CSV 저장
csv_file = DATA_PATH / "processed" / "seoul_gopchang_master_clean.csv"
master_clean.to_csv(csv_file, index=False, encoding='utf-8-sig')
print(f"   💾 CSV 저장: {csv_file}")

print("\n" + "=" * 60)

# ================================================================================
# 📊 정제 결과 검증 및 시각화
# ================================================================================

print("📊 정제 결과 검증 및 시각화")
print("-" * 30)

# 1. 기본 통계
print("1️⃣ 정제 후 기본 통계:")
print(f"   📊 총 상권 수: {len(master_clean):,}개")
print(f"   📋 총 컬럼 수: {len(master_clean.columns)}개")
print(f"   🌙 야간 데이터: {master_clean['야간_유동인구_17_24'].notna().sum():,}개")
print(f"   💰 매출 데이터: {master_clean['총_매출'].notna().sum():,}개")
print(f"   🗺️ 좌표 데이터: {len(master_gdf) if master_gdf is not None else 0:,}개")

# 2. 야간 유동인구 TOP 10 (정제 후)
print(f"\n2️⃣ 야간 유동인구 TOP 10 (정제 후):")
if '야간_유동인구_17_24' in master_clean.columns and '상권_코드_명' in master_clean.columns:
    top_clean = master_clean.nlargest(10, '야간_유동인구_17_24')[
        ['상권_코드_명', '야간_유동인구_17_24', '총_매출']
    ]
    for i, (idx, row) in enumerate(top_clean.iterrows(), 1):
        sales_info = f"{row['총_매출']/1e8:.1f}억원" if pd.notna(row['총_매출']) else "N/A"
        print(f"   {i:2d}. {row['상권_코드_명']:20s}: {row['야간_유동인구_17_24']:7,.0f}명 ({sales_info})")

# 3. 데이터 품질 비교 (정제 전후)
print(f"\n3️⃣ 데이터 품질 비교:")
print(f"   📊 정제 전: 471,240개 → 정제 후: {len(master_clean):,}개")
print(f"   📉 데이터 크기 감소: {(1 - len(master_clean)/471240)*100:.1f}%")
print(f"   ✅ 정상 상권 수 달성: {len(master_clean) == 1650}")

# 4. 시각화
fig_summary = make_subplots(
    rows=2, cols=2,
    subplot_titles=('정제 전후 비교', '야간 유동인구 분포', '매출 효율성', '데이터 완성도'),
    specs=[[{"type": "bar"}, {"type": "histogram"}],
           [{"type": "scatter"}, {"type": "bar"}]]
)

# 4-1. 정제 전후 비교
fig_summary.add_trace(
    go.Bar(x=['정제 전', '정제 후'], y=[471240, len(master_clean)],
           marker_color=['lightcoral', 'lightgreen'],
           text=['471K개', f'{len(master_clean):,}개'],
           textposition='outside'),
    row=1, col=1
)

# 4-2. 야간 유동인구 분포
if '야간_유동인구_17_24' in master_clean.columns:
    night_data = master_clean['야간_유동인구_17_24'].dropna()
    fig_summary.add_trace(
        go.Histogram(x=night_data, nbinsx=30, marker_color='lightblue'),
        row=1, col=2
    )

# 4-3. 매출 효율성 (야간 인구 vs 매출)
if all(col in master_clean.columns for col in ['야간_유동인구_17_24', '총_매출']):
    clean_data = master_clean[['야간_유동인구_17_24', '총_매출']].dropna()
    if len(clean_data) > 0:
        fig_summary.add_trace(
            go.Scatter(x=clean_data['야간_유동인구_17_24'], 
                      y=clean_data['총_매출'],
                      mode='markers', marker_color='orange', opacity=0.6),
            row=2, col=1
        )

# 4-4. 데이터 완성도
completeness = {}
key_cols = ['총_유동인구_수', '야간_유동인구_17_24', '총_매출', '총_상주인구']
for col in key_cols:
    if col in master_clean.columns:
        completeness[col] = (master_clean[col].notna().sum() / len(master_clean)) * 100

if completeness:
    fig_summary.add_trace(
        go.Bar(x=list(completeness.keys()), y=list(completeness.values()),
               marker_color='lightgreen',
               text=[f'{v:.1f}%' for v in completeness.values()],
               textposition='outside'),
        row=2, col=2
    )

fig_summary.update_layout(
    title="🧹 데이터 정제 결과 요약",
    height=800,
    showlegend=False
)

fig_summary.show()

print("\n" + "=" * 60)

# ================================================================================
# 🎯 완료 및 다음 단계
# ================================================================================

print("🎯 데이터 정제 완료!")
print("-" * 30)

print(f"✅ 완료 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

print(f"\n📊 최종 정제 결과:")
print(f"   🗃️  깔끔한 마스터 데이터: {master_clean.shape}")
print(f"   🌙 야간 유동인구: {master_clean['야간_유동인구_17_24'].notna().sum():,}개 상권")
print(f"   💰 매출 데이터: {master_clean['총_매출'].notna().sum():,}개 상권")
print(f"   🗺️ 지리정보: {len(master_gdf) if master_gdf is not None else 0:,}개 상권")
print(f"   💾 저장 파일: seoul_gopchang_master_clean.csv / .gpkg")

print(f"\n🎯 해결된 문제들:")
print(f"   ✅ 업종별 중복 제거: 471K → {len(master_clean):,}개")
print(f"   ✅ 정확한 상권 수: {len(master_clean) == 1650}")
print(f"   ✅ 좌표 정보 복구: {len(master_gdf) if master_gdf is not None else 0}개 상권")
print(f"   ✅ 올바른 TOP 상권 확인")

print(f"\n🚀 이제 준비된 다음 단계:")
print(f"   A. KPI 스코어링 (경쟁밀도, 통합점수) ⭐ 추천")
print(f"   B. 지도 시각화 (Folium + 좌표)")
print(f"   C. 상권별 상세 분석 (TOP 30)")
print(f"   D. 실시간 대시보드")

print("\n" + "=" * 60)
print("🧹 데이터 정제 완료!")
print("🎯 이제 깔끔한 데이터로 본격적인 분석을 시작할 수 있습니다!")